In [ ]:
# we need the following libraries, so let's install them
!pip install boto3
!pip install torchxrayvision
!pip install SimpleITK
!pip install radiomics
!pip install pyradiomics
!pip install torchvision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# check if any cuda-enabled hardware is available
!nvidia-smi
import torch
torch.cuda.is_available()

Fri May 26 00:59:15 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

True

In [ ]:
# importing libraries
from time import time
import numpy as np
import pandas as pd
from glob import glob
import os
import skimage
import SimpleITK as sitk
import skimage, torch, torchvision
import torchxrayvision as xrv
import torchvision
import torchxrayvision as xrv
import matplotlib.pyplot as plt
from tqdm import tqdm

# importing written classes and methods
import sys
from radiomics import featureextractor
from radiomics import firstorder, glcm, glrlm, glszm, shape, shape2D, ngtdm, gldm
import warnings
import logging
warnings.filterwarnings("ignore", category=DeprecationWarning)
# set level for all classes
logger = logging.getLogger("radiomics")
logger.setLevel(logging.ERROR)
# ... or set level for specific class
logger = logging.getLogger("radiomics.glcm")
logger.setLevel(logging.ERROR)

In [ ]:
# pre-defining new directories to save new files
directory_ = '/content/sample_data/CheXpert/'
directory_processed_images = directory_ + 'processed_images'
directory_filterss = directory_ + 'filters'
directory_segments = directory_ + 'segments'
directory_cleaned_targets = directory_ + 'cleaned_targets'

try:
  os.makedirs(directory_processed_images + '/train')
  os.makedirs(directory_processed_images + '/valid')
except:
  pass

try:
  os.makedirs(directory_filterss+ '/train')
  os.makedirs(directory_filterss+ '/valid')
except:
  pass

try:
  os.makedirs(directory_segments+ '/train')
  os.makedirs(directory_segments+ '/valid')
except:
  pass

# try:
#   os.makedirs(directory_cleaned_targets+ '/train')
#   os.makedirs(directory_cleaned_targets+ '/valid')
# except:
#   pass

In [ ]:
class ChestXRaySegmentation:
    def __init__(self, image_size: int = 512, threshold: float = 0.5):
        """

        :param image_size:
        :param threshold:
        """
        self.image_size = image_size
        self.threshold = threshold
        self.resizer = torchvision.transforms.Compose([xrv.datasets.XRayResizer(self.image_size)])
        self.centerer = torchvision.transforms.Compose([xrv.datasets.XRayCenterCrop()])

        def device_selector_cuda(image:np.ndarray):
          return torch.from_numpy(image).cuda(device="cuda:0")

        def device_selector_cpu(image:np.ndarray):
          return torch.from_numpy(image).cpu()

        #----------------------------------------------------------------
        print(f'device is {torch.cuda.is_available()}')
        if torch.cuda.is_available():
          dvc = "cuda:0"
          self.model_segment = xrv.baseline_models.chestx_det.PSPNet().cuda(device=dvc)
          self.device_selector = device_selector_cuda
        else:
          self.model_segment = xrv.baseline_models.chestx_det.PSPNet()
          self.device_selector = device_selector_cpu
        #----------------------------------------------------------------

        self.normalizer = xrv.datasets.normalize
        self.loader = skimage.io.imread
        self.composer = torchvision.transforms.Compose
        self.segment_names = self.model_segment.targets
        self.num_segments = len(self.segment_names)
        self.image = np.zeros(shape=(1, self.image_size, self.image_size))
        self.segment_index = [4,5,8,13]
        #self.segment_index = list(range(14))
    def prep(self, name: str):
        self.image = self.resizer(self.centerer(self.normalizer(self.loader(name)[:,:,0], 255)[np.newaxis, :, :]))
        return self.device_selector(self.image)


    def prep_segment(self, name: str):
        with torch.no_grad(): return self.model_segment(self.prep(name=name))[:,self.segment_index,:,:]

    def get_filters(self, name: str):
        probs = 1 / (1 + torch.exp(-self.prep_segment(name=name)))  # sigmoid function
        return torch.where(probs > self.threshold, 1.0, 0.0).cpu().numpy()

    def get_filter_image(self, name: str):
        probs = 1 / (1 + torch.exp(-self.prep_segment(name=name)))  # sigmoid function
        return torch.where(probs > self.threshold, 1.0, 0.0).cpu().numpy(), self.image, probs.cpu().numpy()

In [ ]:
# reading the labels of images and printing the first row of the data
data_path = '/content/sample_data/CheXpert/targets/metadata.csv'
data = pd.read_csv(data_path)
print(data.shape[0])
data.head(1)

223648


,Path,Patient_id,Study_Number,Sex,Age,Race,Ethnicity,Frontal/Lateral,AP/PA,No Finding,...,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
0,CheXpert-v1.0/train/patient00001/study1/view1_...,1,1,Female,68.0,Other,Non-Hispanic/Non-Latino,Frontal,AP,1,...,0,0,0,0,0,0,0,0,0,1


In [ ]:
# printing all imaging position
pd.unique(data['AP/PA'])

array(['AP', nan, 'PA', 'LL', 'RL'], dtype=object)

In [ ]:
# printing all unique fronal/lateral imaging position
pd.unique(data['Frontal/Lateral'])

array(['Frontal', 'Lateral'], dtype=object)

In [ ]:
# checking the consistency of image views
((data['AP/PA']=='AP') | (data['AP/PA']=='PA')).sum()

191212

In [ ]:
# counting the number of frontal images
(data['Frontal/Lateral']=='Frontal').sum()

191229

In [ ]:
# number of images with AP or PA and frontal labels
(((data['AP/PA']=='AP') | (data['AP/PA']=='PA')) & (data['Frontal/Lateral']=='Frontal')).sum()

191212

In [ ]:
# filtering all lateral images
meta_data_target = pd.read_csv('/content/sample_data/CheXpert/targets/metadata.csv')
meta_data_target.head(2)
N_all_samples = meta_data_target.shape[0]
print(f'total samples are: {N_all_samples}')
mask_lateral_samples = (((meta_data_target['AP/PA']=='AP') | (meta_data_target['AP/PA']=='PA')) & (meta_data_target['Frontal/Lateral']=='Frontal'))
n_images = mask_lateral_samples.sum()
print(f'front image samples are: {n_images}')
truncated_target_data = meta_data_target[mask_lateral_samples]
truncated_target_data.reset_index(inplace=True)
truncated_target_data.head(2)

total samples are: 223648
front image samples are: 191212


,index,Path,Patient_id,Study_Number,Sex,Age,Race,Ethnicity,Frontal/Lateral,AP/PA,...,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
0,0,CheXpert-v1.0/train/patient00001/study1/view1_...,1,1,Female,68.0,Other,Non-Hispanic/Non-Latino,Frontal,AP,...,0,0,0,0,0,0,0,0,0,1
1,1,CheXpert-v1.0/train/patient00002/study2/view1_...,2,2,Female,87.0,"White, non-Hispanic",Non-Hispanic/Non-Latino,Frontal,AP,...,0,-1,-1,0,-1,0,-1,0,1,0


In [ ]:
# getting the name of all variables saved into the loaded dataframe
all_columns = list(truncated_target_data)

In [ ]:
# the name of all segments
segment_names = ['Left Clavicle',
                  'Right Clavicle',
                  'Left Scapula',
                  'Right Scapula',
                  'Left Lung',
                  'Right Lung',
                  'Left Hilus Pulmonis',
                  'Right Hilus Pulmonis',
                  'Heart',
                  'Aorta',
                  'Facies Diaphragmatica',
                  'Mediastinum',
                  'Weasand',
                  'Spine'
                  ]

In [ ]:
# save all target variables in a dictionary format
targets = {}
for col  in all_columns:
  targets.update({col:truncated_target_data[col].values})
np.save(directory_cleaned_targets + '/new_targets.npy',targets)

In [ ]:
# image segmentation and saving processed images, their segments, and the extracted segment filters
segmenter = ChestXRaySegmentation(image_size=512,threshold=0.5)
base_path = '/content/sample_data/CheXpert/images'
n_error = 0
for  n in tqdm(range(n_images)):
  try:
    location = truncated_target_data['Path'][n].split('CheXpert-v1.0')[1]
    sp1 = location.split('/')

    picture_name = sp1[1] + '__' + sp1[2] + '__' + sp1[3] + '__' + sp1[4].split('.jpg')[0]
    filters,image,segments = segmenter.get_filter_image(base_path+location)

    np.save(directory_filterss + '/' +picture_name+'__filters.npy',filters),  np.save(directory_processed_images + '/' +picture_name+'__image.npy',image)
  #,  np.save(directory_segments + '/' +picture_name+'__segments.npy',segments)
  except:
    n_error +=1

device is True


100%|██████████| 191212/191212 [9:34:13<00:00,  5.55it/s]
